In [28]:
import gensim
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import string
import re

In [29]:
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [30]:
labels = pd.read_csv('labels.csv')
train = pd.read_csv('train.csv')

In [31]:
train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [41]:
def data_preprocess(row): #Utility Function
    row = str(row)
    row = row.lower()
    row = re.sub('[^a-zA-Z]', ' ', row)
    row_tokens = word_tokenize(row)
    row = (lemmatizer.lemmatize(word) for word in row_tokens if word not in stopwords)
    row_processed = ' '.join(row_tokens)
    #row_processed = word_tokenize(row_processed)
    return row_processed

In [33]:
train['content'] = train.title + ' ' + train.author

In [34]:
train.content = train.content.apply(data_preprocess)

In [8]:
VECTOR_SIZE = 100
w2v = gensim.models.Word2Vec(train.content,
                            vector_size=VECTOR_SIZE,
                            window=7,
                            min_count=1,
                            epochs=50)

In [9]:
def word2vector(row): #Vector util
    for i,word in enumerate(row):
        if(word in w2v.wv.index_to_key):
            row[i] = np.asarray(w2v.wv[word])
        else:
            row[i] = 0
    row = (np.mean(row, axis=0)).tolist()
    return row

In [10]:
train.content = train.content.apply(word2vector)


c:\Users\Mrulay\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [11]:
df = train.content.apply(pd.Series)

In [12]:
df['label'] = train.label
df = df.dropna()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'label'], df.label, test_size=0.25, random_state=42)

In [14]:
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,0.668894,0.821996,0.321929,-0.958595,-0.065362,0.266006,0.652788,0.271767,-0.350623,1.002040,...,0.284387,-0.893949,-0.111707,0.191977,-0.538335,-0.131207,-0.333054,0.761424,-0.496558,1
1,1.920960,0.345482,0.231933,-0.253391,0.512607,1.082127,0.264062,0.683654,-1.135681,-0.266714,...,-1.031315,0.023972,0.123042,0.192789,-0.387585,-0.872051,-0.190078,-0.804364,0.790919,0
2,0.314369,-0.375896,-0.572484,-1.250595,-0.549824,0.169462,-0.712321,0.910779,0.053339,1.258737,...,0.535325,-0.833016,-1.276946,-0.610670,-0.160201,-1.404328,0.057129,-0.390538,-1.122155,1
3,0.911460,-0.481384,1.140141,-0.587918,-0.270819,0.470909,0.091845,1.092736,0.559045,-0.111708,...,-0.222222,0.713855,-0.371541,0.305381,-0.600119,0.382483,0.274246,0.301541,-0.467800,1
4,0.406740,0.528648,0.077141,0.125268,-0.574987,0.807347,-0.040396,0.195821,-0.580574,0.252547,...,0.412093,0.314349,-0.520602,0.275927,0.296997,0.693326,0.456611,0.074491,0.638095,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20795,0.872482,0.519724,0.607144,-0.356767,-0.758656,1.125608,0.159298,0.135372,-0.565150,1.111445,...,1.837866,-1.225709,-0.543379,0.624787,-0.713669,-0.768185,-1.705534,-0.496877,0.195294,0
20796,-0.667301,0.748833,-0.442047,0.308489,0.790697,0.130301,0.011303,0.976521,-0.222467,0.327054,...,0.494846,1.092259,0.097544,0.597184,-0.594477,0.240805,-0.504205,0.579502,0.063569,0
20797,-0.384687,0.048353,-0.281788,-0.471755,0.098482,0.745488,-0.108597,1.134383,0.011775,0.032442,...,0.093460,0.933567,-0.449934,0.657906,0.298438,-0.161413,-0.360772,-0.708034,0.676193,0
20798,0.911517,-0.265299,-0.313204,-0.373709,-0.503450,0.757700,-0.399392,0.723084,0.260886,-0.369040,...,-0.385994,0.171494,-0.407586,0.607357,0.155787,-1.198669,0.184077,0.593833,-0.694102,1


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
linear_clf = RandomForestClassifier()

linear_clf.fit(X_train, y_train)
prediction2 = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test, prediction2)
print("accuracy:   %0.3f" % score)
metrics.confusion_matrix(y_test, prediction2)

accuracy:   0.960


array([[2469,  105],
       [ 102, 2517]], dtype=int64)

In [16]:
test = pd.read_csv('test.csv')
test['labels'] = labels.label

In [17]:
test['content'] = test.title + ' ' + test.author

In [18]:
test.content = test.content.apply(data_preprocess)

In [19]:
test.content = test.content.apply(word2vector)

c:\Users\Mrulay\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
c:\Users\Mrulay\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [20]:
df = test.content.apply(pd.Series)
df['label'] = labels.label
df = df.dropna()

In [21]:
prediction2 = linear_clf.predict(df.loc[:, df.columns != 'label'])
score = metrics.accuracy_score(df.label, prediction2)
print("accuracy:   %0.3f" % score)
metrics.confusion_matrix(df.label, prediction2)

accuracy:   0.642


array([[1541,  797],
       [1060, 1789]], dtype=int64)

# LSTM

In [89]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [90]:
train.text = train.text.apply(data_preprocess)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.25, random_state=42)

In [92]:
max_words = 20000
max_len = 350
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [93]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [94]:
model = RNN()
model.summary()
model.compile(loss='mean_squared_error',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 350)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 350, 50)           1000000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [95]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
98/98 [==============================] - 3s 24ms/step - loss: 0.1248 - accuracy: 0.8255 - val_loss: 0.0513 - val_accuracy: 0.9311
Epoch 2/10
98/98 [==============================] - 2s 19ms/step - loss: 0.0456 - accuracy: 0.9408 - val_loss: 0.0552 - val_accuracy: 0.9266
Epoch 3/10
98/98 [==============================] - 2s 19ms/step - loss: 0.0282 - accuracy: 0.9643 - val_loss: 0.0496 - val_accuracy: 0.9388
Epoch 4/10
98/98 [==============================] - 2s 19ms/step - loss: 0.0192 - accuracy: 0.9769 - val_loss: 0.0614 - val_accuracy: 0.9196
Epoch 5/10
98/98 [==============================] - 2s 19ms/step - loss: 0.0128 - accuracy: 0.9845 - val_loss: 0.0562 - val_accuracy: 0.9311
Epoch 6/10
98/98 [==============================] - 2s 17ms/step - loss: 0.0116 - accuracy: 0.9861 - val_loss: 0.0618 - val_accuracy: 0.9256
Epoch 7/10
98/98 [==============================] - 2s 17ms/step - loss: 0.0061 - accuracy: 0.9930 - val_loss: 0.0570 - val_accuracy: 0.9362
Epoch 8/10
98

In [96]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [97]:
accr = model.evaluate(test_sequences_matrix,y_test)

163/163 [==============================] - 1s 7ms/step - loss: 0.0602 - accuracy: 0.9325


In [98]:
test = pd.read_csv('test.csv')
test['labels'] = labels.label

In [99]:
test.text = test.text.apply(data_preprocess)

In [101]:
test_sequences = tok.texts_to_sequences(test.text)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [102]:
accr = model.evaluate(test_sequences_matrix,test.labels)

163/163 [==============================] - 1s 7ms/step - loss: 0.3731 - accuracy: 0.6185
